In [3]:
#import diskcache
import os
import xarray
import s3fs

/Users/aimeebarciauskas/miniconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
# cache = dc.Cache(directory='./diskcache')
# @cache.memoize(tag='diskcache_xarray_open_dataset')
def xarray_open_dataset(src_path: str) -> xarray.Dataset:
    """Open dataset."""
    xr_open_args: Dict[str, Any] = {
        "decode_coords": "all",
        "decode_times": False,
        "chunks": None,
        "engine": "h5netcdf"
    }
    fs = s3fs.S3FileSystem()
    file_handler = fs.open(src_path)
    ds = xarray.open_dataset(file_handler, **xr_open_args)
    return ds

In [5]:
%%time
src_path = 's3://nasa-veda-scratch/cmip6-staging/prXin/ACCESS-CM2/prXin-ACCESS-CM2-ssp126.nc'
ds1 = xarray_open_dataset(src_path)

/Users/aimeebarciauskas/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


CPU times: user 988 ms, sys: 409 ms, total: 1.4 s
Wall time: 2.68 s


In [6]:
%%time
src_path = 's3://veda-data-store-staging/rechunking/prXin-ACCESS-CM2-ssp126_compressed.nc'
ds2 = xarray_open_dataset(src_path)

CPU times: user 1.67 s, sys: 318 ms, total: 1.98 s
Wall time: 15.2 s


In [ ]:
%%time
loaded1 = ds1.load()

In [ ]:
%%time
loaded2 = ds2.load()

In [ ]:
display_variable = 'prXin_1'
ds_rechunked = ds1.chunk(chunks={'time': 1, 'lat': 600, 'lon': 1440})
ds_rechunked[display_variable]

In [ ]:
%%time
target_file_name = "prXin-ACCESS-CM2-ssp126_small_chunks_compressed.nc"
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in ds1.data_vars}
loaded1.to_netcdf(target_file_name, encoding=encoding)

In [ ]:
print(f"Compressed netCDF size {os.path.getsize(target_file_name)/1000000} MB")

In [ ]:
fs = s3fs.S3FileSystem()
new_target = f"s3://nasa-veda-scratch/cmip6-staging/{target_file_name}"
fs.put(target_file_name, new_target)

In [ ]:
%%time
ds3 = xarray_open_dataset(new_target)

In [ ]:
%%time
ds3.load()